### Factorization Machine

In [4]:
class FeaturesEmbedding(torch.nn.Module):

    def __init__(self, field_dimensions, embedding_dimensions):
        
        super().__init__()
        
        fields_count = sum(field_dimensions)
        
        self.embedding = torch.nn.Embedding(fields_count, embedding_dimensions)
        
        field_offsets = np.cumsum(field_dimensions)
        
        self.embedding_offsets = np.array((0, *field_offsets[:-1]), dtype=int)
        
        torch.nn.init.xavier_uniform_(self.embedding.weight.data)
        
        # self.embedding.weight.data = self.embedding.weight.data / 10000
        
        # print("fields_count:", fields_count)
        # print("field_offsets:", field_offsets)        
        # print("self.embedding_offsets:", self.embedding_offsets)
        # print("self.embedding.weight.data:", self.embedding.weight.data)              
        
    def forward(self, embedding_inputs):
        
        one_hot_offsets = embedding_inputs.new_tensor(self.embedding_offsets).unsqueeze(0)

        one_hot_positions = embedding_inputs + one_hot_offsets
        
        embedding_weights = self.embedding(one_hot_positions)
        
#         print("embedding_inputs:", embedding_inputs)        
#         print("one_hot_positions:", one_hot_positions)        
#         print("embedding_weights:", embedding_weights)
        
        return embedding_weights

In [4]:
class FactorizationMachine(torch.nn.Module):

    def __init__(self, reduce_sum=True):
        
        super().__init__()
        
        self.reduce_sum = reduce_sum

    def forward(self, weights):

        square_of_sum = torch.sum(weights, dim=1) ** 2
        sum_of_squares = torch.sum(weights ** 2, dim=1)
        
        prediction = square_of_sum - sum_of_squares           
        
        if self.reduce_sum:
            prediction = torch.sum(prediction, dim=1, keepdim=True)
        
        prediction = 0.5 * prediction

        # print("square_of_sum:", square_of_sum)    
        # print("sum_of_squares:", sum_of_squares)               
        # print("prediction:", prediction)        
        
        return prediction

In [7]:
class CustomFactorizationMachine(torch.nn.Module):

    def __init__(self, field_dimensions, embedding_dimensions):
        
        super().__init__()
        
        self.embedding = FeaturesEmbedding(field_dimensions, embedding_dimensions)
        # self.linear = FeaturesLinear(field_dimensions)
        self.fm = FactorizationMachine(reduce_sum=True)
        

    def forward(self, inputs):
        
        # linear_out = self.linear(inputs)
        
        embedding_weights = self.embedding(inputs)
        
        fm_predictions = self.fm(embedding_weights)
        
        # result = (linear_out + fm_predictions).squeeze(1)
        
        result = fm_predictions.squeeze(1)
        
        # sigmoid_result = torch.sigmoid(squeezed_result)
        
        # print("inputs:", inputs)    
        # print("embedding_weights:", embedding_weights)               
        # print("fm_predictions:", fm_predictions)  
        # print("result:", result)
        
        return result    

### Other Models

In [ ]:
# class FeaturesLinear(torch.nn.Module):

#     def __init__(self, field_dims, output_dim=1):
        
#         super().__init__()
        
#         self.fc = torch.nn.Embedding(sum(field_dims), output_dim)
#         self.bias = torch.nn.Parameter(torch.zeros((output_dim,)))
#         self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=int)

#     def forward(self, x):

#         x = x + x.new_tensor(self.offsets).unsqueeze(0)
        
#         return torch.sum(self.fc(x), dim=1) + self.bias

In [8]:
# class CustomDeepFactorizationMachineModel(torch.nn.Module):

#     def __init__(self, field_dims, embed_dim, mlp_dims, dropout):
        
#         super().__init__()
        
#         self.linear = FeaturesLinear(field_dims)
#         self.fm = FactorizationMachine(reduce_sum=True)
#         self.embedding = FeaturesEmbedding(field_dims, embed_dim)
#         self.embed_output_dim = len(field_dims) * embed_dim
#         self.mlp = MultiLayerPerceptron(self.embed_output_dim, mlp_dims, dropout)

#     def forward(self, x):

#         embed_x = self.embedding(x)
        
#         result = self.linear(x) + self.fm(embed_x) + self.mlp(embed_x.view(-1, self.embed_output_dim))
        
#         squeezed_result = result.squeeze(1)
        
#         sigmoid_result = torch.sigmoid(squeezed_result)
        
#         return squeezed_result

In [ ]:
# class MultiLayerPerceptron(torch.nn.Module):

#     def __init__(self, input_dim, embed_dims, dropout, output_layer=True):
        
#         super().__init__()
        
#         layers = list()
        
#         for embed_dim in embed_dims:
#             layers.append(torch.nn.Linear(input_dim, embed_dim))
#             layers.append(torch.nn.BatchNorm1d(embed_dim))
#             layers.append(torch.nn.ReLU())
#             layers.append(torch.nn.Dropout(p=dropout))
#             input_dim = embed_dim
            
#         if output_layer:
#             layers.append(torch.nn.Linear(input_dim, 1))
            
#         self.mlp = torch.nn.Sequential(*layers)

#     def forward(self, x):

#         return self.mlp(x)

In [ ]:
# class CustomLinearRegression(torch.nn.Module):
    
#     def __init__(self, input_dimensions, output_dimensions):
        
#         super().__init__()
        
#         self.linear = torch.nn.Linear(input_dimensions, 
#                                       output_dimensions)

#     def forward(self, x): 
        
#         result = self.linear(x.float())
        
#         squeezed_result = result.squeeze(1)
        
#         return squeezed_result